# Backpropagation - Kettenregel
Das Training von Neuronalen Netzen (NN), die auf gradientenbasierten Optimierungsalgorithmen basieren, erfolgt in zwei Hauptschritten:

- Forward Propagation - hier berechnen wir die Ausgabe des NN bei gegebenen Eingaben

- Backward Propagation - hier berechnen wir die Gradienten der Ausgabe in Bezug auf die Eingaben, um die Gewichte zu aktualisieren

In [9]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from torch.autograd import Variable

1. Computational Graph: Nehmen wir an, wir wollen die folgende Reihe von Operationen durchführen, um unser Ergebnis $r$ zu erhalten

    $u = x^2$

    $v = u+y$

    $w = z*v$

    $r = w^2$

   >Zeichnen Sie den Graph zu den obigen Operationen

3. Um dieses Konzept greifbarer zu machen, nehmen wir einige Zahlen für unsere Berechnung. Zum Beispiel: 

   $x=1$

   $y=2$

   $z=4$

3. Berechnen Sie den "Forward Pass" und die folgenden Gradienten mit Hilfe der Kettenregel.

      
$\frac{ \partial r }{ \partial z} = $

$\frac{ \partial r }{ \partial y} = $
   
$\frac{ \partial r }{ \partial x} = $

In [4]:
x = 1.0
y = 2.0
z = 4.0

# forward pass
u = x**2
v = u+y
w = z*v
r = w**2
print(f'r = {r}')

# backward pass
# TODO Start

dr_dz = 2*w*v
dr_dy = 2*w*z
dr_dx = 2*w*z*2*x

# TODO End

print(f'dr/dz = {dr_dz}')
print(f'dr/dy = {dr_dy}')
print(f'dr/dx = {dr_dx}')

r = 144.0
dr/dz = 72.0
dr/dy = 96.0
dr/dx = 192.0


5. Verwendung von pytorch zur Berechnung von Gradienten. Stellen Sie sicher, dass die Variable einen Tensor verwendet und setzen Sie `requires_grad=True`

In [5]:
# TODO Start
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = torch.tensor(4.0, requires_grad=True)
# TODO End

# forward pass
u = x**2
v = u+y
w = z*v
r = w**2
print(f'r = {r}')

# backward pass
r.backward()

print(f'dr/dz = {z.grad}')
print(f'dr/dy = {y.grad}')
print(f'dr/dx = {x.grad}')
# TODO End

r = 144.0
dr/dz = 72.0
dr/dy = 96.0
dr/dx = 192.0


## Rückwärtspropagierung - MLP mit Numpy

In dieser Aufgabe implementieren Sie bitte die Vorwärts- und Rückwärtspropagation von MLP mit Numpy. 

>Beziehen Sie sich bei der Implementierung auf die Seiten 30 und 32 der Folien. 

>Achten Sie bei der Implementierung der Backpropagation auf die Form der Matrix. 

>Die Gewichte und der Gradient_w jeder Schicht müssen die gleiche Form haben, und dasselbe gilt für die Biases und den Gradient_b.

In [ ]:
class CustomNet_Numpy:

    def __init__(self, input_size, output_size, hidden_sizes):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.biases = []
        self.weights = []

        self.sizes = [input_size] + hidden_sizes + [output_size]
        self.num_layers = len(self.sizes)

        for i in range(len(self.sizes)-1):
            weight = np.random.normal(size=(self.sizes[i+1], self.sizes[i]))
            bias = np.random.normal(size=(self.sizes[i+1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def sigmoid(self, z):
        return 1.0/(1.0 + np.exp(-z))
    
    def sigmoid_prime(self, z):
        return sigmoid(z) * (1-sigmoid(z))

    def forward(self, x):
        # feedforward
        activation = x
        activations = [x] 
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(activation, w.T)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
        return activations, zs   

    def backward(self, x, y):
        # gradient
        gradient_b = [np.zeros(b.shape) for b in self.biases]
        gradient_w = [np.zeros(w.shape) for w in self.weights]
        # forward pass
        activations, zs = self.forward(x)
        
        # derivative of loss function
        d_loss = self.loss_derivative(activations[-1], y)

        # backward pass
        
        gradient_b[-1] = d_loss
        gradient_w[-1] = np.outer(d_loss, activations[-2])
        
        for l in np.arange(2, self.num_layers):
            pass
        
        return gradient_b, gradient_w
    
    def loss_func(self, output_activations, y):
        return 0.5*torch.mean((output_activations - y)**2)
    
    def loss_derivative(self, output_activations, y):
        return torch.mean(output_activations - y)

In [ ]:
# Test example
input_size = 10
output_size = 1
hidden_sizes = [20, 30, 40]

net = CustomNet_Numpy(input_size, output_size, hidden_sizes)

# input data
x = np.random.randn(100, input_size)
y = np.random.randn(100, output_size)

# feedforward and backpropagation
grad_b, grad_w = net.backward(x, y)

for i in range(len(grad_w)):
    print("Gradient of layer {}: gw {}".format(i+1, grad_w[i].shape))

## Backpropagation - MLP mit pytorch

In der obigen Aufgabe spüren Sie vielleicht die Komplexität der manuellen Berechnung der Backpropagation, obwohl es sich nur um ein sehr einfaches MLP handelt. 
Wenn das Netzwerk komplexer wird, wird die manuelle Berechnung der Backpropagation unkontrollierbar. Hier spielt PyTorch seine Vorteile aus, da der Benutzer nur eine API aufrufen muss, um den Gradienten aller Parameter zu berechnen. Führen Sie dazu die folgenden Schritte aus

>Berechnen Sie zuerst der Loss-Wert

>Setzen Sie die Gradienten des Netzes auf Null mit `zero_grad()` vor dem nächsten "Backward Pass"

>Berechnen Sie die Gradienten in abhängigkeit des aktuellen Loss-Wertes mit `backward()`

>Speicher Sie für jedes Layer die Gradienten

In [17]:
torch.manual_seed(42)

class CustomNet_Torch(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes):
        super(CustomNet_Torch, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.layers = []
        
        sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(sizes)-1):
            layer = nn.Linear(sizes[i], sizes[i+1])
            nn.init.xavier_uniform_(layer.weight)
            self.layers.append(layer)
        
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = torch.sigmoid(layer(out))
        return out
    
    def backward(self, x, y):
        x = Variable(torch.Tensor(x))
        y = Variable(torch.Tensor(y))
        
        # feedforward
        out = self.forward(x)
        
        # calculate loss
        loss = self.loss_func(out,y)
        
        # backpropagation
        self.zero_grad()
        loss.backward()
        
        # Compute the gradient for each weight and bias
        gradients = []
        for i in range(len(self.layers)):
            gradients.append(self.layers[i].weight.grad)
        
        return loss.item(), gradients

    def loss_func(self, output_activations, y):
        return 0.5*torch.mean((output_activations - y)**2)


In [18]:
# Test example
input_size = 10
output_size = 1
hidden_sizes = [20, 30, 40]

net = CustomNet_Torch(input_size, output_size, hidden_sizes)

# input data
x = torch.randn(100, input_size)
y = torch.randn(100, output_size)

# feedforward and backpropagation
loss, gradients = net.backward(x, y)

print("Loss:", loss)
for i in range(len(gradients)):
    print("Gradient of layer {}: {}".format(i+1, gradients[i].shape))

Loss: 0.5432382225990295
Gradient of layer 1: torch.Size([20, 10])
Gradient of layer 2: torch.Size([30, 20])
Gradient of layer 3: torch.Size([40, 30])
Gradient of layer 4: torch.Size([1, 40])
